In [1]:
%matplotlib inline

In [21]:
import datetime
import pandas as pd
import numpy as np
import datetime

In [22]:
import catboost
from catboost import CatBoostClassifier, Pool, cv
import hyperopt

In [23]:
import tqdm

In [24]:
from sklearn.pipeline import Pipeline
import sklearn.model_selection
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import model_selection , metrics   #Additional scklearn functions

## import and parameters

In [166]:
imput_strategy = 1

In [167]:
train = pd.read_pickle('../data/merged_data/train.pkl')
test = pd.read_pickle('../data/merged_data/test.pkl')

In [132]:
#train = train.sample(frac=0.1)

In [133]:
# preprocess
y_train = train['target'].values

# to try:
* optimize aucroc with keras and batch methods
* SVM / rank SVM (https://github.com/rdipietro/pyrvm)
* try separate models on separate variables + stacking

## Features : 
* NLP on COMMENTAIRE_BI
* Extract options from OPTION (pas sur que ce soit util, peut être juste check la qualité)
* features from history



* check for intervention in test without contracts and to handle them
* add combinations of categorical features (see paribas example on catboost site)
* select only best features (feature_importance ?)
* try to put random nas in train for handling missing contracts

## Fails:
* one model with selectKbest on modalities vs one model on dimensionality reduction with group rare modalities
    * dimensionality reduction don't bring much value here (mb MCA)
    * select k best fails with 5k categories, which is not enough to be interesting

In [168]:
drop = ['INSTANCE_ID', #460k modalities, not usable as a feature
        'INCIDENT_NUMBER']
drop_atm = [#'AUTEUR_INCIDENT', # 2088 modalities
            'TYPE_VOIE',
#            'NATURE_CODE', # 313 modalities, need to be splitted in 5 modalities
#            'MARQUE_LIB', # 167 modalities
#            'OPTION', # 80 modalities, extract options
            #'MODELE_CODE', # 10k modalities
#            'COMMENTAIRE_BI', # NLP 400k modalities
             #'RESOURCE_ID', # 4033 modalities
            'CODE_POSTAL', # 5800 modalities (only get first 2 numbers ?)
            'L2_ORGA_CODE_POSTAL', # 147 modalities (might be redondent with L2_ORGANISATION_ID)
#            'L2_ORGANISATION_ID' #151 modalities
            'L2_ORGA_VILLE', # 146, might be redondent with other organisation variables
#            'RACHAT_CODE' # 312 modalities (try binarising ?)         
#            'CODE_INSTALLATION' # 17 modalities
           ]

In [169]:
#train.drop(drop + drop_atm + ['target'], axis=1, inplace=True)
train.drop(drop + drop_atm , axis=1, inplace=True)
test.drop(drop + drop_atm, axis=1, inplace=True)

In [170]:
categoricals = list(train.columns[train.dtypes == 'category'])

### imputation of missing data

TODO: try imputing test based on test values, not train <br>
TODO: try diffrent strategy on imputing datas from contract since missing are present only in test set<BR>
TODO: try creating data with missing contract in train sample and do not fill the missing in test<br>
TODO: try imputing specific na value for categorical variables

In [171]:
def impute_selected_variables(df, test, categ, quanti, dates):
    _df = df.copy()
    _test = test.copy() if test is not None else None
    
    replace = _df[categ].mode()
    replace_values = {k:v.iloc[0] for k,v in replace.items()}
    _df.fillna(replace_values, inplace=True)

    replace_quanti = _df[quanti].mean()
    _df.fillna(replace_quanti, inplace=True)

    _df[dates] = _df[dates].fillna(method='pad')
    
    if test is not None:
        
        _test.fillna(replace_values, inplace=True)
        _test.fillna(replace_quanti, inplace=True)
        _test[dates] = _df[dates].fillna(method='pad')
    
    return _df, _test

In [172]:
#replace = train[categoricals].mode()
#replace_values = {k:v.iloc[0] for k,v in replace.items()}
def impute_contract_variables(df):
    _df = df.copy()
    
    for var in categ_contract:
        try:
            _df[var] = _df[var].fillna('NAN')
        except ValueError as e:
            _df[var] = _df[var].cat.add_categories(['NAN'])
            _df[var] = _df[var].fillna('NAN') 

    _df[quanti_contract] = _df[quanti_contract].fillna(-9999)
    _df[date_contract] = _df[date_contract].fillna(datetime.datetime(1970, 1, 1))
    return _df

In [173]:
categoricals = list(train.columns[train.dtypes == 'category'])
quantitative = ['NB_PASSAGE', 'POINTS_FIDEL', 'CONTRAT_TARIF', 'PRIX_FACTURE']
dates = list(train.columns[train.dtypes == 'datetime64[ns]'])

In [174]:
contract_variables = [ 'UPD_DATE', 'DATE_DEBUT', 'DATE_FIN', 'STS_CODE', 'OPTION', 'FORMULE', 'CONTRAT_TARIF', 'PRIX_FACTURE']

In [175]:
#impute without contract
categ_to_impute = list(set(categoricals) - set(contract_variables))
quanti_to_impute = list(set(quantitative) - set(contract_variables))
date_to_impute = list(set(dates) - set(contract_variables))

#impute contract
categ_contract = list(set(categoricals).intersection(set(contract_variables)))
quanti_contract = list(set(quantitative).intersection(set(contract_variables)))
date_contract = list(set(dates).intersection(set(contract_variables)))

#train and test are filled with values taken from train
#contract and other variables are imputed together
if imput_strategy == 1:
    train, test = impute_selected_variables(train, test, categoricals, quantitative, dates)

#train and test are filled with values taken on their own values
#contract and other variables are imputed together
if imput_strategy == 2:
    train, _ = impute_selected_variables(train, None, categoricals, quantitative, dates)
    test, _ = impute_selected_variables(test, None, categoricals, quantitative, dates)

#train and test are filled with values taken from train
#contract and other variables are imputed separatly (need to import some NAN in train set)
if imput_strategy == 3:
    train, test = impute_selected_variables(train, test, categ_to_impute, quanti_to_impute, date_to_impute)
    train = impute_contract_variables(train)
    test = impute_contract_variables(test)

# train and test are filled on their own values
# contract and other variables are imputed separatly (need to import some NAN in train set)
if imput_strategy == 4:
    train, _ = impute_selected_variables(train, None, categ_to_impute, quanti_to_impute, date_to_impute)
    test, _ = impute_selected_variables(test, None, categ_to_impute, quanti_to_impute, date_to_impute)
    train = impute_contract_variables(train)
    test = impute_contract_variables(test)

### Feature ingineering

In [ ]:
#to adapt
def cleanText(s):
    s = s.str.lower()                         # Convert to lowercase
    s = s.str.replace(r'<.*?>', ' ')          # Remove HTML characters
    s = s.str.replace('\'', '')               # Remove single quotes ' 
    s = s.str.replace('-', '')                # Remove dashes -
    s = s.str.replace(r'[^a-zA-Z]', ' ')      # Remove non alpha characters
    s = s.map(lambda s: ' '.join(s.split()))  # Remove extra whitespae
    s = s.str.strip()                         # Remove whitespace at start and end
#    s = s.apply(lambda x: removeStopwords(x)) # Remove stopwords
return s

In [176]:
def commentaire_bi(df):
    _df = df.copy()
    
    _df.COMMENTAIRE_BI = _df.COMMENTAIRE_BI.str.upper()
    COMMENTAIRE_BI_vc = _df.COMMENTAIRE_BI.value_counts()
    common_commentaire_bi = COMMENTAIRE_BI_vc[COMMENTAIRE_BI_vc > 100].index
    _df['COMMENTAIRE_BI_common'] = _df.COMMENTAIRE_BI.where(_df.COMMENTAIRE_BI.isin(common_commentaire_bi), "Rare")
    
    _df['nb_char_commentaire'] = [len(txt) for txt in _df.COMMENTAIRE_BI]
    _df['nb_mots_commentaire'] = [len(txt.split()) for txt in _df.COMMENTAIRE_BI]
    _df['has_number_commentaire'] = [any(char.isdigit() for char in txt) for txt in _df.COMMENTAIRE_BI]
    _df['is_empty_commentaire'] = [(txt == '.') for txt in _df.COMMENTAIRE_BI]
    _df.drop('COMMENTAIRE_BI', axis=1, inplace=True)
    
    return _df

In [177]:
def nature_code_split(df):
    _df = df.copy()
    nature_code_splitted = [nc.split('-') for nc in df.NATURE_CODE]
    nature_code_df = pd.DataFrame(nature_code_splitted, columns=['nc_1', 'nc_2', 'nc_3', 'nc_4', 'nc_5'])
    nature_code_df.fillna('-1', inplace=True)
    for nc_i in ['nc_1', 'nc_2', 'nc_3', 'nc_4', 'nc_5']:
        nature_code_df[nc_i] = nature_code_df[nc_i].astype('category')
    
    #_df.drop('NATURE_CODE', axis=1, inplace=True)
    _df = _df.merge(nature_code_df, left_index=True, right_index=True)
    return _df

In [178]:
# todo: use dt series accessor
def add_dates_features(data):
    data['age_installation'] = (data['CRE_DATE_GZL'] - data['INSTALL_DATE']).dt.days // 365
    data['mois_appel'] = data['CRE_DATE_GZL'].map(lambda x: x.month)
    data['joursemaine_appel'] = data['CRE_DATE_GZL'].map(lambda x: x.isoweekday()) #integer, might be considered categorical
    data['jour_appel'] = data['CRE_DATE_GZL'].map(lambda x: x.day)
    data['mois_intervention'] = data['SCHEDULED_START_DATE'].map(lambda x: x.month)
    data['joursemaine_intervention'] = data['SCHEDULED_START_DATE'].map(lambda x: x.isoweekday()) #integer, might be considered categorical
    data['jour_intervention'] = data['SCHEDULED_START_DATE'].map(lambda x: x.day)
    data['duree_avant_intervention'] = (data['SCHEDULED_START_DATE'] - data['CRE_DATE_GZL']).dt.days
    data['duree_prevue'] = (data['SCHEDULED_END_DATE'] - data['SCHEDULED_START_DATE']).dt.days
    data['temps_depuis_debut_contrat'] = (data['CRE_DATE_GZL'] - data['DATE_DEBUT']).dt.days
    data['temps_jusqua_fin_contrat'] = (data['CRE_DATE_GZL'] - data['DATE_FIN']).dt.days  #souvent nan ? (mettre 0)
    data['temps_depuis_maj_contrat'] = (data['CRE_DATE_GZL'] - data['UPD_DATE']).dt.days 

    data.drop(['CRE_DATE_GZL', 'INSTALL_DATE', 'SCHEDULED_START_DATE', 'SCHEDULED_END_DATE', 'DATE_DEBUT', 'DATE_FIN', 'UPD_DATE'], axis=1, inplace=True)
    return data

In [179]:
#mots en particulier
#créer un dictionnaire et compter (count et garder les must ? voir la diff entre les cas + et - ?)

In [180]:
# feature engineering

# (history) temps depuis dernière visite (pas forcément dispo sur le test)
# (history) déjà eu une casse sur ce matériel
# (history) temps depuis dernière casse
# (history) la dernière visite date de moins de 6 mois
# (history) nb interventions faires par la ressource
# (history) temps depuis la première intervention de la ressource
# (contract history) nb de fois que le contrat a été mis à jour sur les X dernières années

In [181]:
def add_features_from_file(df, csv_file):
    features = pd.read_csv(csv_file)
    features['canceled_pred'] = features['canceled_pred'].astype(bool)
    df = df.join(features)
    return df

In [182]:
train = commentaire_bi(train)
train = nature_code_split(train)
train = add_dates_features(train)
train = add_features_from_file(train, 'features_canceled_train.csv')

In [183]:
test = commentaire_bi(test)
test = nature_code_split(test)
test = add_dates_features(test)
test = add_features_from_file(test, 'features_canceled_test.csv')

In [184]:
#drop variables
#drop = ['joursemaine_appel', 'USAGE_LOCAL', 'nc_4', 'is_empty_commentaire', 'duree_prevue', 'nc_1']

#train.drop(drop, axis=1,inplace=True)
#test.drop(drop, axis=1,inplace=True)

## Get variables where test set has modalities which are not in train set

In [106]:
categoricals = list(train.columns[train.dtypes == 'category'])
var_with_new_categ = []
for cat in categoricals:
    if len(set(test[cat]) - set(train[cat])) > 0:
        var_with_new_categ.append(cat)

# drop categories in test which are not in train

In [107]:
for cat in categoricals:
    train[cat] = train[cat].cat.remove_unused_categories()

In [108]:
class CategoriesDroper(BaseEstimator, TransformerMixin):
    '''Drop categories which are not in the train set'''

    def __init__(self, columns):
        self.columns = columns
        self.categories_dict = dict()
        
    def fit(self, df, y=None):
        self.categories_dict = {column: df[column].cat.categories for column in self.columns}
        return self

    def transform(self, df):
        _df = df.copy()
        for column in self.columns:
            _df[column] = _df[column].cat.set_categories(self.categories_dict[column])
            try:
                _df[column] = _df[column].fillna('NAN')
            except ValueError as e:
                _df[column] = _df[column].cat.add_categories(['NAN'])
                _df[column] = _df[column].fillna('NAN') 
                print(e,'with ', column)
        
        return _df

In [109]:
cd = CategoriesDroper(categoricals)
cd.fit(train)
test = cd.transform(test)

fill value must be in categories with  INCIDENT_TYPE_ID
fill value must be in categories with  TYPE_BI
fill value must be in categories with  MILLESIME
fill value must be in categories with  PROBLEM_CODE
fill value must be in categories with  AUTEUR_INCIDENT
fill value must be in categories with  GRAVITE
fill value must be in categories with  RESOURCE_ID
fill value must be in categories with  TYPE_OCC
fill value must be in categories with  RACHAT_CODE
fill value must be in categories with  NATURE_CODE
fill value must be in categories with  MARQUE_LIB
fill value must be in categories with  MODELE_CODE
fill value must be in categories with  PAYS
fill value must be in categories with  STOP_PHONING
fill value must be in categories with  CODE_GEN_EQUIPEMENT
fill value must be in categories with  CODE_FONCTION
fill value must be in categories with  CODE_ENERGIE
fill value must be in categories with  CODE_INSTALLATION
fill value must be in categories with  CODE_SPECIFICATION
fill value must b

# Modelisation

### split train / val

In [85]:
#for item in categoricals:
#    train[item] = train[item].cat.codes +1

In [86]:
# split train in train, cv (will be replaced by cross validation for parameters tuning)
# stratify ?
X_train_train, X_train_val, y_train_train, y_train_val = sklearn.model_selection.train_test_split(train, y_train, random_state=101)

In [87]:
pos_neg_ratio =  sum(y_train==0) / sum(y_train)

In [88]:
#combinations_ctr for list of variables ?
model = CatBoostClassifier(
    iterations=200,
#    one_hot_max_size=3,
    learning_rate=0.16,
#    depth=8,
    eval_metric="AUC",
    random_seed=42,
    od_type='Iter',
    od_wait=40,
    use_best_model=True
#    scale_pos_weight=pos_neg_ratio
)

In [89]:
categoricals = train.select_dtypes(include=['category', 'bool', 'object']).columns
categorical_features_indices = [X_train_train.columns.get_loc(cat) for cat in categoricals]

In [90]:
model.fit(
    X_train_train, y_train_train,
    cat_features=categorical_features_indices,
    eval_set=(X_train_val, y_train_val),
    logging_level='Verbose'  # you can uncomment this for text output
)

0:	test: 0.7048973	best: 0.7048973 (0)	total: 3.18s	remaining: 10m 32s
1:	test: 0.7106869	best: 0.7106869 (1)	total: 6.6s	remaining: 10m 53s
2:	test: 0.7126851	best: 0.7126851 (2)	total: 9.72s	remaining: 10m 38s
3:	test: 0.7140945	best: 0.7140945 (3)	total: 12.9s	remaining: 10m 32s
4:	test: 0.7145079	best: 0.7145079 (4)	total: 16s	remaining: 10m 25s
5:	test: 0.7151936	best: 0.7151936 (5)	total: 19.5s	remaining: 10m 29s
6:	test: 0.7163818	best: 0.7163818 (6)	total: 22.5s	remaining: 10m 19s
7:	test: 0.7168471	best: 0.7168471 (7)	total: 25.6s	remaining: 10m 14s
8:	test: 0.7173209	best: 0.7173209 (8)	total: 28.7s	remaining: 10m 9s
9:	test: 0.7178285	best: 0.7178285 (9)	total: 32.1s	remaining: 10m 9s
10:	test: 0.7188165	best: 0.7188165 (10)	total: 35s	remaining: 10m 1s
11:	test: 0.7194177	best: 0.7194177 (11)	total: 38.5s	remaining: 10m 3s
12:	test: 0.7198280	best: 0.7198280 (12)	total: 42.1s	remaining: 10m 5s
13:	test: 0.7201591	best: 0.7201591 (13)	total: 45.8s	remaining: 10m 8s
14:	test:

113:	test: 0.7348478	best: 0.7348478 (113)	total: 7m 42s	remaining: 5m 48s
114:	test: 0.7348490	best: 0.7348490 (114)	total: 7m 45s	remaining: 5m 44s
115:	test: 0.7348533	best: 0.7348533 (115)	total: 7m 49s	remaining: 5m 40s
116:	test: 0.7348791	best: 0.7348791 (116)	total: 7m 54s	remaining: 5m 36s
117:	test: 0.7349988	best: 0.7349988 (117)	total: 7m 58s	remaining: 5m 32s
118:	test: 0.7350095	best: 0.7350095 (118)	total: 8m 3s	remaining: 5m 28s
119:	test: 0.7350209	best: 0.7350209 (119)	total: 8m 7s	remaining: 5m 25s
120:	test: 0.7350309	best: 0.7350309 (120)	total: 8m 11s	remaining: 5m 21s
121:	test: 0.7350452	best: 0.7350452 (121)	total: 8m 16s	remaining: 5m 17s
122:	test: 0.7350449	best: 0.7350452 (121)	total: 8m 20s	remaining: 5m 13s
123:	test: 0.7350537	best: 0.7350537 (123)	total: 8m 25s	remaining: 5m 9s
124:	test: 0.7350681	best: 0.7350681 (124)	total: 8m 29s	remaining: 5m 5s
125:	test: 0.7350637	best: 0.7350681 (124)	total: 8m 33s	remaining: 5m 1s
126:	test: 0.7350723	best: 0.7

199:	test: 0.7686368	best: 0.7686409 (198)	total: 14m 41s	remaining: 0us (nlp, lr=0.31)
199:	test: 0.7686109	best: 0.7686109 (199)	total: 14m 17s	remaining: 0us (nlp, lr=0.31, 9 variables dropped)
199:	test: 0.7729298	best: 0.7730353 (190)	total: 22m 13s	remaining: 0us (nlp, lr=0.31, 6 variables dropped, depth=8)
199:	test: 0.7576144	best: 0.7576257 (197)	total: 15m 42s	remaining: 0us(nlp, lr=0.31, weighted, one_hot=3)

### Feature importance

In [ ]:
importances=model.feature_importances_

In [ ]:
indices = np.argsort(model.feature_importances_)[::-1]

print("Feature ranking:")

for f in range(train.shape[1]):
    print("%d. %s (%f)" % (f + 1, train.columns[indices[f]], model.feature_importances_[indices[f]]))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(train.shape[1]), importances[indices],
       color="r", align="center")
plt.xticks(range(train.shape[1]), indices)
plt.xlim([-1, train.shape[1]])
plt.show()

### Hyperparameter tunning

In [ ]:
categoricals = train.select_dtypes(include=['category', 'bool', 'object']).columns
categorical_features_indices = [train.columns.get_loc(cat) for cat in categoricals]

#### GridSearch

In [ ]:
from sklearn.grid_search import GridSearchCV

In [ ]:
#what to do with cat features
params = {'depth': [4, 7, 10],
          'learning_rate' : [0.20, 0.31, 0.40],
         'l2_leaf_reg': [1, 4, 9],
         'iterations': [300]}
cb = CatBoostClassifier()
cb_model = GridSearchCV(cb, params, scoring="roc_auc")
cb_model.fit(train, y_train, cb__cat_features=categorical_features_indices)

#### hyperopt

In [ ]:
def hyperopt_objective(params):
    model = CatBoostClassifier(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        iterations=int(params['iterations']),
        border_count=int(params['border_count']),
        eval_metric='AUC',
        random_seed=42,
        logging_level='Silent'
        #od_type='Iter',
        #od_wait=40
    )
    
    cv_data = cv(
        Pool(train, y_train, cat_features=categorical_features_indices),
        model.get_params(),
        stratified=True,
        fold_count=5
    )
    best_AUC = np.max(cv_data['test-AUC-mean'])
    print(params, 'best_AUC: ', best_AUC)
    
    return 1 - best_AUC # as hyperopt minimises

In [ ]:
#param space example for xgboost
#    space = {
#             'n_estimators' : hp.quniform('n_estimators', 100, 1000, 1),
#             'eta' : hp.quniform('eta', 0.025, 0.5, 0.025),
#             'max_depth' : hp.quniform('max_depth', 1, 13, 1),
#             'min_child_weight' : hp.quniform('min_child_weight', 1, 6, 1),
#             'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
#             'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
#             'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05),
#             'num_class' : 9,
#             'eval_metric': 'mlogloss',
#             'objective': 'multi:softprob',
#             'nthread' : 6,
#             'silent' : 1
#}

In [ ]:
from numpy.random import RandomState

params_space = {
    #'l2_leaf_reg': hyperopt.hp.loguniform('l2_leaf_reg', -1, np.log(50)),
    'l2_leaf_reg': hyperopt.hp.choice('l2_leaf_reg', [11.47871028241772]),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-1, 8e-1),
     #'learning_rate': hyperopt.hp.choice('learning_rate', [0.31]),
    'iterations': hyperopt.hp.quniform('iterations', 250, 1000, 1),
    #'iterations': hyperopt.hp.choice('iterations', [250]),
    #'depth': hyperopt.hp.quniform('depth', 3, 9, 1),
    'depth': hyperopt.hp.choice('depth', [7]),
#    'ctr_border_count': hyperopt.hp.quniform('ctr_border_count', 32, 255, 1),
    #'border_count': hyperopt.hp.quniform('border_count', 16, 255, 1)
    'border_count': hyperopt.hp.choice('border_count', [213])
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=8,
    trials=trials,
    rstate=RandomState(123)
    
)

print(best)

#round 1
{'depth': 7.0, 'l2_leaf_reg': 6.687638927479829} best_AUC:  0.764339203919555
{'depth': 8.0, 'l2_leaf_reg': 2.238434810070511} best_AUC:  0.7641940842160905
{'depth': 9.0, 'l2_leaf_reg': 15.61615505112418} best_AUC:  0.7656179177534072
{'depth': 7.0, 'l2_leaf_reg': 11.47871028241772} best_AUC:  0.766724797438951
{'depth': 6.0, 'l2_leaf_reg': 2.0946141383866816} best_AUC:  0.7641940842160905
{'depth': 5.0, 'l2_leaf_reg': 3.409700409045487} best_AUC:  0.7656396435014416
{'l2_leaf_reg': 11.47871028241772, 'learning_rate': 7.0}

#round 2
{'border_count': 199.0, 'depth': 7, 'iterations': 250, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.31} best_AUC:  0.7654693707749893
{'border_count': 213.0, 'depth': 7, 'iterations': 250, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.31} best_AUC:  0.7662068818039395
{'border_count': 105.0, 'depth': 8, 'iterations': 250, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.31} best_AUC:  0.7639026943327258
{'border_count': 238.0, 'depth': 7, 'iterations': 250, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.31} best_AUC:  0.764574925126635
{'border_count': 22.0, 'depth': 8, 'iterations': 250, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.31} best_AUC:  0.7643064612539859
{'border_count': 103.0, 'depth': 8, 'iterations': 250, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.31} best_AUC:  0.7632638136706521
{'border_count': 225.0, 'depth': 8, 'iterations': 250, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.31} best_AUC:  0.7653002122083675
{'border_count': 212.0, 'depth': 8, 'iterations': 250, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.31} best_AUC:  0.7649391813119792
{'border_count': 85.0, 'depth': 8, 'iterations': 250, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.31} best_AUC:  0.7649657537494025
{'border_count': 251.0, 'depth': 8, 'iterations': 250, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.31} best_AUC:  0.7649243873489558
{'border_count': 213.0, 'depth': 0, 'iterations': 0, 'l2_leaf_reg': 0, 'learning_rate': 0}

#round 3
{'border_count': 213, 'depth': 7, 'iterations': 693.0, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.6096210638782021} best_AUC:  0.767539056284941
{'border_count': 213, 'depth': 7, 'iterations': 526.0, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.701085638831484} best_AUC:  0.767228088670764
{'border_count': 213, 'depth': 7, 'iterations': 822.0, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.7947626751796161} best_AUC:  0.7647414699770261
{'border_count': 213, 'depth': 7, 'iterations': 775.0, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.571434077994127} best_AUC:  0.7686875566115616
{'border_count': 213, 'depth': 7, 'iterations': 516.0, 'l2_leaf_reg': 11.47871028241772, 'learning_rate': 0.4938304425560929} best_AUC:  0.7680238623609622


In [ ]:
#check quality to expect
model = CatBoostClassifier(
    l2_leaf_reg=int(best['l2_leaf_reg']),
    learning_rate=best['learning_rate'],
    iterations=500,
    eval_metric='AUC',
    random_seed=42,
    logging_level='Silent'
)
cv_data = cv(Pool(X, y, cat_features=categorical_features_indices), model.get_params())

In [ ]:
print('Precise validation AUC score: {}'.format(np.max(cv_data['test-AUC-mean'])))

### Make submission

In [150]:
train = train.drop(['canceled_pred', 'canceled_proba_pred'], axis=1)
test = test.drop(['canceled_pred', 'canceled_proba_pred'], axis=1)

In [151]:
categoricals = train.select_dtypes(include=['category', 'bool', 'object']).columns
categorical_features_indices = [train.columns.get_loc(cat) for cat in categoricals]

In [152]:
pos_neg_ratio =  sum(y_train==0) / sum(y_train==1)

In [155]:
params = {
#          'border_count': 128, 
#          'one_hot_max_size': 5,
#          'depth': 7,
#          'depth': 5,
          'iterations': 50.0,
#          'l2_leaf_reg': 11.47871028241772, 
          'learning_rate': 0.15,
#          'scale_pos_weight': pos_neg_ratio
          'thread_count': 3
         }

#random_strength (default 1)

In [156]:
model = CatBoostClassifier(
    **params,
    eval_metric="AUC",
    od_type='Iter',
    od_wait=40
)

model.fit(train, y_train, cat_features=categorical_features_indices)

0:	total: 5.32s	remaining: 4m 20s
1:	total: 10.3s	remaining: 4m 8s
2:	total: 15.3s	remaining: 3m 59s
3:	total: 19.9s	remaining: 3m 48s
4:	total: 24.6s	remaining: 3m 41s
5:	total: 28.9s	remaining: 3m 31s
6:	total: 33.5s	remaining: 3m 25s
7:	total: 37.8s	remaining: 3m 18s
8:	total: 42.6s	remaining: 3m 14s
9:	total: 47.3s	remaining: 3m 9s
10:	total: 52.7s	remaining: 3m 6s
11:	total: 57.7s	remaining: 3m 2s
12:	total: 1m 2s	remaining: 2m 58s
13:	total: 1m 8s	remaining: 2m 55s
14:	total: 1m 13s	remaining: 2m 50s
15:	total: 1m 17s	remaining: 2m 45s
16:	total: 1m 23s	remaining: 2m 41s
17:	total: 1m 29s	remaining: 2m 38s
18:	total: 1m 33s	remaining: 2m 33s
19:	total: 1m 39s	remaining: 2m 29s
20:	total: 1m 46s	remaining: 2m 27s
21:	total: 1m 53s	remaining: 2m 24s
22:	total: 1m 58s	remaining: 2m 19s
23:	total: 2m 4s	remaining: 2m 14s
24:	total: 2m 9s	remaining: 2m 9s
25:	total: 2m 15s	remaining: 2m 5s
26:	total: 2m 21s	remaining: 2m
27:	total: 2m 27s	remaining: 1m 56s
28:	total: 2m 34s	remaining:

In [106]:
#save model to disk
#model.save_model('catboost_model.dump')
#load
#model = CatBoostClassifier()
#model.load_model('catboost_model.dump')

In [157]:
submisstion = model.predict_proba(test)[:, 1]

In [158]:
filename = str(datetime.datetime.now())[:-7] + '_submission.csv'
sub = pd.Series(submisstion, name='target')
sub.to_csv(filename, index=False, header=True)

all parameters free and 200 iterations : 0.67
all parameters free and 200 iterations and delete ressource_id : 0.67
200 iterations + learning rate = 0.31 : 0.694
50 iterations + learning rate = 0.15 : 0.7057
200 iterations + learning rate = 0.15 : 0.695
'' + deletion of a all id like variables : 0.684
'' +  add cancellation results : 0.698 (vs 0.73 in val)
200 iterations + learning rate = 0.15 + l2 = 11.5 :


### tests on cross validation

In [ ]:
pos_neg_ratio =  sum(y_train==0) / sum(y_train==1)

categoricals = train.select_dtypes(include=['category', 'bool', 'object']).columns
categorical_features_indices = [train.columns.get_loc(cat) for cat in categoricals]

model = CatBoostClassifier(
    border_count=128, 
    one_hot_max_size= 5,
    depth= 5,
    iterations= 200.0, 
    l2_leaf_reg= 11.478, 
    scale_pos_weight= pos_neg_ratio,
    eval_metric='AUC'
)

In [ ]:
cv_data = cv(
    Pool(train, y_train, cat_features=categorical_features_indices),
    model.get_params(),
    stratified=True,
    fold_count=5
)

#0.707 on test set

In [ ]:
cv_data.plot(y=['test-AUC-mean', 'train-AUC-mean'])

In [ ]:
cv_data.plot(y=['test-Logloss-mean', 'train-Logloss-mean'])

### train with some randomly added na for contract variables

In [110]:
var_with_new_categ_not_in_contract = [var for var in var_with_new_categ if var not in categ_contract]

In [111]:
quanti_contract = ['PRIX_FACTURE', 
                   'CONTRAT_TARIF',
                   'temps_depuis_debut_contrat',
                   'temps_jusqua_fin_contrat', 
                   'temps_depuis_maj_contrat']

In [112]:
for var in categ_contract + var_with_new_categ_not_in_contract:
    try:
        train.loc[:, var] = train[var].cat.add_categories(['NAN'])
    except ValueError as e:
        next

In [113]:
categoricals = train.select_dtypes(include=['category', 'bool', 'object']).columns
categorical_features_indices = [train.columns.get_loc(cat) for cat in categoricals]

In [114]:
models = []

#np.random.seed(42)
for i in tqdm.tqdm_notebook(range(10)):
    X_train = train.copy()
    
    #randomly put missing contract in train data
    _idx = np.random.choice(X_train.index, size=X_train.shape[0]//20, replace=False)
    X_train.loc[_idx, categ_contract] = 'NAN'
    X_train.loc[_idx, quanti_contract] = -9999
    
    #randomly put missing in variables with new categories in test
    for var in var_with_new_categ_not_in_contract:
        _idx = np.random.choice(X_train.index, size=X_train.shape[0]//20, replace=False)
        X_train.loc[_idx, var] = 'NAN'

    model = CatBoostClassifier(
        iterations=200,
        depth=8,
        border_count=128,
        learning_rate=0.30,
#        l2_leaf_reg= 11.478,
        eval_metric='AUC',
        verbose=False
    )
    
    model.fit(X_train, y_train, cat_features=categorical_features_indices)
    models.append(model.copy())

In [115]:
predictions = []
for _model in tqdm.tqdm_notebook(models):
    predictions.append(_model.predict_proba(test)[:,1])
    
submisstion = np.vstack(predictions).T.mean(axis=1)

In [116]:
filename = str(datetime.datetime.now())[:-7] + '_submission.csv'
sub = pd.Series(submisstion, name='target')
sub.to_csv(filename, index=False, header=True)

In [48]:
predictions = []
for _model in tqdm.tqdm_notebook(models):
    predictions.append(_model.predict_proba(X_train_val)[:,1])
    
predictions = np.vstack(predictions).T.mean(axis=1)

# 150 arbres : 0.7624871381817937
# 200 arbres : .7638860824056132
print(sklearn.metrics.roc_auc_score(y_train_val, predictions))


0.7638860824056132


In [ ]:
#prediction = np.hstack([predictions, predictions2]).mean(axis=1)

## LightGBM

In [ ]:
import lightgbm

In [ ]:
model_gbm = lightgbm.LGBMClassifier(
    seed=np.random.randint(10**10),
    n_estimators=500,
    max_depth=10,
    num_leaves=200,
    min_data_in_leaf=100,
    subsample=0.8,
    colsample_bytree=0.8,
    #is_unbalance=True,
    objective='binary'
)

In [ ]:
model_gbm.fit(X_train_train, y_train_train, 
              eval_set=[(X_train_val, y_train_val)], 
              eval_metric='auc', 
              categorical_feature=categorical_features_indices)

## H2O random forest
http://docs.h2o.ai/h2o/latest-stable/h2o-docs/grid-search.html#grid-search-in-python

TODO: assembling categorical features (see netstack tutorial)
TODO: compare LB to val score
TODO: remove cancelled binary
TODO: hyper parameter optimization

In [159]:
import h2o

In [161]:
h2o.init(nthreads=2, max_mem_size = "8G")             #specify max number of bytes. uses all cores by default.
h2o.remove_all() #clean slate, in case cluster was already running

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_161"; Java(TM) SE Runtime Environment (build 1.8.0_161-b12); Java HotSpot(TM) 64-Bit Server VM (build 25.161-b12, mixed mode)
  Starting server from /home/julien/anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp9qoibaju
  JVM stdout: /tmp/tmp9qoibaju/h2o_julien_started_from_python.out
  JVM stderr: /tmp/tmp9qoibaju/h2o_julien_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Paris
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.5
H2O cluster version age:,24 days
H2O cluster name:,H2O_from_python_julien_rk9xb4
H2O cluster total nodes:,1
H2O cluster free memory:,7.111 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [160]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
help(H2OGradientBoostingEstimator)
help(h2o.import_file)

Help on class H2OGradientBoostingEstimator in module h2o.estimators.gbm:

class H2OGradientBoostingEstimator(h2o.estimators.estimator_base.H2OEstimator)
 |  Gradient Boosting Machine
 |  
 |  Builds gradient boosted trees on a parsed data set, for regression or classification.
 |  The default distribution function will guess the model type based on the response column type.
 |  Otherwise, the response column must be an enum for "bernoulli" or "multinomial", and numeric
 |  for all other distributions.
 |  
 |  Method resolution order:
 |      H2OGradientBoostingEstimator
 |      h2o.estimators.estimator_base.H2OEstimator
 |      h2o.model.model_base.ModelBase
 |      h2o.utils.backward_compatibility.BackwardsCompatibleBase
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, **kwargs)
 |      Construct a new model instance.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  balance_classe


Help on function import_file in module h2o.h2o:

import_file(path=None, destination_frame=None, parse=True, header=0, sep=None, col_names=None, col_types=None, na_strings=None, pattern=None)
    Import a dataset that is already on the cluster.
    
    The path to the data must be a valid path for each node in the H2O cluster. If some node in the H2O cluster
    cannot see the file, then an exception will be thrown by the H2O cluster. Does a parallel/distributed
    multi-threaded pull of the data. The main difference between this method and :func:`upload_file` is that
    the latter works with local files, whereas this method imports remote files (i.e. files local to the server).
    If you running H2O server on your own maching, then both methods behave the same.
    
    :param path: path(s) specifying the location of the data to import or a path to a directory of files to import
    :param destination_frame: The unique hex key assigned to the imported file. If none is given, a key

In [194]:
train.target = train.target.astype(bool)

In [195]:
#covtype_df = h2o.import_file(os.path.realpath("../data/covtype.full.csv"))
#convert df to H2O df type
covtype_df = h2o.H2OFrame(train)

/home/julien/anaconda3/lib/python3.6/site-packages/h2o/utils/shared_utils.py:177: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [213]:
#split the data as described above
X_train, X_valid, X_test = covtype_df.split_frame([0.60, 0.20], seed=1234)

#Prepare predictors and response columns

covtype_X = covtype_df.col_names     #last column is Cover_Type, our desired response variable 
covtype_X.remove('target')
covtype_y = 'target'

In [224]:
rf_v1 = H2ORandomForestEstimator(
    model_id="rf_v1",
    ntrees=50,
    stopping_rounds=2,
    score_each_iteration=True,
    seed=1000000)

In [225]:
rf_v1.train(covtype_X, covtype_y, training_frame=X_train, validation_frame=X_valid)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [226]:
sklearn.metrics.roc_auc_score(X_test['target'].as_data_frame(), rf_v1.predict(X_test[covtype_X])[2].as_data_frame())

drf prediction progress: |████████████████████████████████████████████████| 100%


0.7171321991348156

In [ ]:
rf_v2 = H2ORandomForestEstimator(
    model_id="rf_covType_v2",
    ntrees=200,
    max_depth=30,
    stopping_rounds=2,
    stopping_tolerance=0.01,
    score_each_iteration=True,
    seed=3000000)
rf_v2.train(covtype_X, covtype_y, training_frame=train, validation_frame=valid)

In [ ]:
rf_v2.hit_ratio_table(valid=True)

## WOE + LR
https://github.com/patrick201/information_value/blob/master/src/information_value.py
